In [ ]:
!pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

class VulnScanner:
    def __init__(self, url):
        self.url = url
        self.session = requests.Session()

    def check_headers(self):
        print("🛡️ Checking security headers...")
        resp = self.session.get(self.url)
        missing = []
        headers = {
            "X-Frame-Options": "Prevents clickjacking",
            "X-XSS-Protection": "Enables XSS filter",
            "Content-Security-Policy": "Helps prevent XSS",
            "Strict-Transport-Security": "Enforces HTTPS",
        }
        for header in headers:
            if header not in resp.headers:
                missing.append((header, headers[header]))
        if missing:
            print("[!] Missing security headers:")
            for h, desc in missing:
                print(f"  - {h}: {desc}")
        else:
            print("[+] All common security headers present.")

    def check_csrf_in_forms(self):
        print("\n🧾 Checking for CSRF tokens in forms...")
        try:
            resp = self.session.get(self.url)
            soup = BeautifulSoup(resp.text, "html.parser")
            forms = soup.find_all("form")
            if not forms:
                print("[*] No forms found.")
                return
            for i, form in enumerate(forms):
                inputs = form.find_all("input")
                has_token = any("csrf" in (inp.get("name") or "").lower() for inp in inputs)
                action = form.get("action")
                full_action = urljoin(self.url, action) if action else self.url
                if has_token:
                    print(f"[+] Form {i+1} ({full_action}) includes CSRF token.")
                else:
                    print(f"[!] Form {i+1} ({full_action}) is missing CSRF token.")
        except Exception as e:
            print("Error checking CSRF tokens:", e)

    def test_reflected_xss(self):
        print("\n🔍 Testing for reflected XSS...")
        test_payload = "<script>alert(1)</script>"
        try:
            test_url = self.url + "?q=" + test_payload
            resp = self.session.get(test_url)
            if test_payload in resp.text:
                print(f"[!] Possible reflected XSS at: {test_url}")
            else:
                print("[+] No reflected XSS detected.")
        except Exception as e:
            print("Error testing for XSS:", e)

    def run_all(self):
        print(f"\n🔎 Scanning: {self.url}")
        self.check_headers()
        self.check_csrf_in_forms()
        self.test_reflected_xss()

In [3]:
# Replace with your test site (must be one you own or have permission to scan!)
scanner = VulnScanner("http://example.com")
scanner.run_all()


🔎 Scanning: http://example.com
🛡️ Checking security headers...
[!] Missing security headers:
  - X-Frame-Options: Prevents clickjacking
  - X-XSS-Protection: Enables XSS filter
  - Content-Security-Policy: Helps prevent XSS
  - Strict-Transport-Security: Enforces HTTPS

🧾 Checking for CSRF tokens in forms...
[*] No forms found.

🔍 Testing for reflected XSS...
[+] No reflected XSS detected.
